# I2R-NUS Submission to Oriental Language Recognition AP16-OL7 Challenge

# Abstract 
本文对 AP16-17 OLR 挑战中 李海洲 等 提交的最佳系统进行详细描述 以及分析.  

system = subsys-ivector  + subsys- GMM-SVM   

Feature: Bottleneck feature 
Central: language-dependent UBM GMM-SVM , and ivector polynomials expansion(多项式膨胀) SVM .  
Fusion:  FoCal Toolkit 用来fusion subsystem.   

significant improvement , EER 0.4%(经验误差) 1.09%(测试误差)   




# 1 Introduction 
该方案是两重子系统熔合.  
First, 使用BNF 替代MFCC-SDC,  用来提取 ivector 和 GMM supervector, BNF 特征就会导致巨大的性能提升.   
GMM-SVM supervector 方案中, 使用 Language dependent GMM策略, 来提取 Language dependent supervector.  
(这里并不是 Language denpendent UBM!!! 是GMM!!)
Second, 使用 pair-wise SVM 分类器 训练模型, 并提升 pair-wise 识别性能.    
打分级别校准 以及 FoCal 熔合 ivector 和 GMM-supervector 打分 形成最终打分结果.  
(那么 ivector, GMM-supervector 的 UBM 是否使用的也是 language dependet ?)   

SECTION 2 , DataBase .
SECTION 3 , front-end Feature  extraction. 
SECTION 4 , 描述分类器.  
SECTION 5 , 描述得分校准, 以及 FoCal 熔合.  
SECTION 6,7, 结果.  





# 3 Front end Feature extraction
## 3.1 BNF 
语种识别的性能, 通过使用一个鲁棒性的特征 能够提升巨大的性能, 鲁棒性的特征 是 能够捕获语种特性, 同时抑制 类似噪音 和 说话人特性的特征.  
例如BNF 就是这样的一种特征. 这里概述的描述下 BNF提取器的训练过程.   
1 language-independent GMM-HMM 系统 训练来识别 语言无关的13361个senone labels.  Feature: MFCC
2 当获得得到一个GMM-HMM的 senone label对齐, 就使用DNN 替代GMM-HMM.   DNN Feature: FBANK + PITCH, 并加上帧拼接.   
3 最终 倒数第二层 是一个瓶颈层, 作为特征输出. 

## 3.2  BNF VAD
我们使用谱减法, 来帮助实现 静音检测, BNF特征 看来还是需要 VAD 的, 但是具体怎么实现呢?
   
**HOW [8] ---------------------------------------------------------------**   


# 4 SYSTEM DESCRIPTIONS 
## 4.1 ivector SVM  
1 64 BNF     
2 VAD    
3 512 UBM   
4 600 ivector   
5 600 ivector 然后 通过 polynomial 二阶多项式基函数 扩展为 180901 维, ivector-polynomials    
**HOW-----------------------------------------------------------------------**   
6 减 均值 , unit-formed归一化 得到 ivector-poly-unit.  
$$ iv_{new} = \frac{iv - \mu_{element} }{|| iv - \mu_{element}||} $$
7 SVM 分类.  


## 4.2 GMM-SVM System 
1 64 BNF 
2 训练 7个(Language Cnt) 512 GMM-UBM(Language-denpendent)   
3 GMM-UBM的均值构成超矢量$S_i$, 正则化     
$$ S_{i_{new}} = \frac{S_i - \mu_{element} }{|| S_i - \mu_{element}||} $$  
4 每个utt 根据对应的512GMM-UBM, 生成一个 32768 = 64(BNF) x 512(UBM Component Cnt) 的 supervector .   
5 最终得到 7 个集合的语种相关的 GMM supervector.    
6 然后训练SVM
7 测试时, 一个test utt 会得到 7个supervector, 然后分别得到对应得分, 最终得分均值作为GMM-SVM 子系统得分? 这里有问题, 这样的得分没有语种信息啊.   
**HOW----------------------------------------------------------------------**   

## 4.3 SVM Classifier 
pair-wise 策略 训练 SVM 语种模型.  
**HOW[9]**    
pair-wise 是 多个类别目标,  两两目标之间进行判别分类.  那么对于 7个分类目标, 最终要训练 21个SVM  21=(7 x 6)/2 (排列组合)    
    
Train:  
$$ f_{m,i} = SVM_{train}(\Phi_m | \Phi_i ), i != m  \tag{4}$$
   
Test:   
test supervector $S_i$, 对所有 21 个SVM分类器进行打分评价, 对 k-th 个语言的打分为 所有包含k-th language 的 6个分类器,中打分最小的得分值. 就是$S_i$ 识别为k语言的最可能得分(该得分最终是对比最不可能语言的得分比)    
**HOW[10]---------------------------------------------------------------**    
$$ Socre_{i-for-kth} = min \Big( f_{k, j}(S_i)\Big) , j=1..N, j!=k \tag{5}$$ 


# 5 Scorce Calibration and Fusion 
## 5.1 Scorce Calibration and Fusion 
本任务, 使用了 1-vs-rest 策略训练不同的子系统, 因此 M 个子系统, N个语种 最终会有 MN个分类测试, 测试一个$S_i$属于某个语种, 每个$S_i$测试具有M个分类器. 在[7]中 测试了多种不同的 score Calibration 和 Fusion 技术. 最终, 虽然线性熔合方法 简单, 但是线性融合方法 高效, 实现简单, 并且 一般效果很好.   
Score Fusion:    
$$ S_l = \sum_{m=1}^{M} \alpha_m x s(m, l) + \beta \tag{6} $$ 
$\alpha_m$ 是 m 子系统的权重, $\beta$ 是n维偏移向量, l 代表一个trial测试.   最终熔合参数 通过 FoCal Multi-Class Toolkit[5] 最小化 $C_{llr}$ 获得.  
  
  
最终每个语种的测试结果, 根据bayes原理, 通过对数似然比[11]计算得到:   
$$ S_{DET}(L_n) = log\Big[\frac{exp(s_n)}{(\frac{1}{N-1})  \sum_{k!=n} exp(s_k)}\Big] $$
$$ S_{DET}(L_n) = s_n - log\Big(\frac{1}{N-1} \sum_{k!=n} exp(s_k) \Big) $$
因为 $ C_{miss} = C_{fa} = 1 , P_{target} = 0.5 $ , 最终检测门限就是0, 当$S_{DET}(L_n)$ > 0, 说明为对应语种.    
本次实验, 也试验了 进行后端得分处理 与 不进行的比较.  

# 7 Conclusion 
1 BNF 是一个提升重点   
2 scores Calibration 和 FoCal 熔合策略 也是一个提升重点.  
